In [52]:
using LinearAlgebra, Statistics
using LaTeXStrings, Plots, Interpolations, Roots, Optim, Random, Parameters
using Optim: maximum, maximizer

# Stochastic Optimal Growth Model

### The problem

Let the sequence of shock process $\{\xi_i\}$ to be i.i.d., with the common distribution denoted by $\phi$.

The household's problem is to maximize
$$ \mathbb{E}_0 \sum_{t=0}^{\infty} \beta^t u(c_t) $$
subject to 
$$ y_{t+1} = f(y_t - c_t) \xi_{t+1}, \; 0\leq c_t \leq y_t $$

Note: one assumes full depreciation of capital each period.


### Recursive formulation
The problem can be casted in the following recurisve formulation:
$$ V(y) = \max_{0\leq c\leq y} \; u(c) + \mathbb{E} \beta V(f(y - c) \xi)   \tag{1} $$

### Method 1: Value function iteration
One can directly iterate on $V$ in the functional equation (1) via:
$$ Tv(y) = \max_{0\leq c\leq y} \; u(c) + \mathbb{E} \beta v(f(y - c) \xi)   \tag{2} $$
We seek a fixed point $v^{*} = Tv^{*}$.

### Method 2: Policy function iteration
Applying the first order condition to (1) yields:
$$ u'(c^{*}) = \mathbb{E} \beta V'(f(y - c^{*}) \xi) f'(y-c^{*}) \xi $$
which, combined with the envelop condition, $ V'(y) = u'(c^{*}(y)) $, yields the Euler equation:
$$ u'(c^{*}(y)) = \mathbb{E} \beta u'(c^{*}(f(y - c^{*}(y)) \xi )) f'(y-c^{*}(y)) \xi \tag{3} $$
To see why the envelop condition holds, note that the problem is equivalent to the one in which one chooses $k$, and $c = y - k$.

Equation (3) can be viewed as a functional equation in the policy function $c(y)$. To find the policy function that satisfies (3), one can come up with the following iterative procedure:
$$ u'(\sigma(y)) = \mathbb{E} \beta u'(\sigma(f(y - c) \xi) ) f'(y-c) \xi \tag{4} $$
The operator $K$ (Coleman operator) takes a given policy function $\sigma$ as given and solve for $c$ for each given $y$. This gives $K\sigma$ in the policy space. We seek a fixed point $\sigma^{*} = K\sigma^{*}$


## Pseudo-codes
We assume a lognormal distribution of the income shock. A simple way to evaluate the integral is by performing a Monte Carlo draw and evaluate the mean.

### Method 1: Fitted value function iteration

> 1. Choose grid points $\{y_1, ... y_n\}$. Choose an initial value function $\{v_1, ... v_n\}$. Use linear interpolation to evaluate the value function at points between the two grid points.
> 2. For each $1\leq i \leq n$, maximize the RHS of (1) using a non-linear optimizer. Keep track of both the optimal value attained as well as the optimal policy $c_i$. Update $v_i$ with the attained optimal value.
> 3. Iterate until convergence as specified by some threshold.

### Method 2: Time iteration/Policy function iteration (Coleman 1990)
> 1. Choose grid points $\{y_1, ... y_n\}$. Choose an initial policy function $\{\sigma_1, ... \sigma_n\}$.  Use linear interpolation to evalute $\sigma$ off the grid points.
> 2. For each $1\leq i \leq n$, solve for $c$ in equation (4) using a non-linear solver. Update the policy function on the grid points by $c_1, ..., c_n$.
> 3. Iterate until cnovergence as specified by some threshold.


### Method 3: Endogenous grid method (Carroll 2006)

In [ ]:
# ========== Method 1: value function iteration ==========

# a bellman operator
function T_VFI(v0; params, c_lb=1e-10)
    (; y_grid, ξ, β, u, f) = params

    v1 = similar(v0)    # new value function
    c  = similar(v0)    # optimal policy function
    v0_func = LinearInterpolation(y_grid, v0, extrapolation_bc=Line())

    for i, y in enumerate(y_grid)
        res = maximize(c -> -(u(c; params) + β * mean(v0_func.(f(y - c) .* ξ))), c_lb, y)
        v1[i] = minimum(res)
        c[i]  = minimizer(res)
    end

    return (;v=v1, c)
end

# ========== Method 2: value function iteration ==========

# a Coleman operator
function K_PFI(σ0; params, c_lb=1e-10)
    (; y_grid, ξ, β, u, du, f, df) = params

    σ1 = similar(σ0)    # new policy function
    σ0_func = LinearInterpolation(y_grid, σ0, extrapolation_bc=Line())

    for i, y in enumerate(y_grid)
        fn2solve(c) = du(σ0_func(y)) - mean(β * du(σ0_func.(f(y - c) .* ξ)) .* df(y - c) .* ξ)
        σ1[i] = find_zero(σ0_func, c_lb, y)
    end

    return (;σ1)
end

# ========== Method 3: endogenous grid method ==========

# a Coleman operator
function K_PFI(σ0; params, c_lb=1e-10)
    (; y_grid, ξ, β, u, du, f, df) = params

    σ1 = similar(σ0)    # new policy function
    σ0_func = LinearInterpolation(y_grid, σ0, extrapolation_bc=Line())

    for i, y in enumerate(y_grid)
        fn2solve(c) = du(σ0_func(y)) - mean(β * du(σ0_func.(f(y - c) .* ξ)) .* df(y - c) .* ξ)
        σ1[i] = find_zero(σ0_func, c_lb, y)
    end

    return (;σ1)
end


In [70]:
[1,2,3] .* 3 .* [1,1,1]

3-element Vector{Int64}:
 3
 6
 9